In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import pandas as pd
import numpy as np
import re
import nltk

In [3]:
df=pd.read_csv('/kaggle/input/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [5]:
df=df.drop(columns=['id','title','author'],axis=1)

In [6]:
df=df.dropna(axis=0)

In [7]:
from nltk.corpus import stopwords

stop=stopwords.words('english')

def cleanText(txt):
    txt=txt.lower()
    txt=' '.join([word for word in txt.split() if word not in (stop)])
    txt=re.sub('[^A-Za-z0-9\s]',' ',txt)
    txt=re.sub('\s',' ',txt)
    return txt

In [8]:
df['text']=df['text'].apply(cleanText)

In [9]:
df['text'][0]

'house dem aide  didn t even see comey s letter jason chaffetz tweeted darrell lucus october 30  2016 subscribe jason chaffetz stump american fork  utah   image courtesy michael jolley  available creative commons by license  apologies keith olbermann  doubt worst person world week fbi director james comey  according house democratic aide  looks like also know second worst person well  turns comey sent now infamous letter announcing fbi looking emails may related hillary clinton s email server  ranking democrats relevant committees didn t hear comey  found via tweet one republican committee chairmen  know  comey notified republican chairmen democratic ranking members house intelligence  judiciary  oversight committees agency reviewing emails recently discovered order see contained classified information  long letter went out  oversight committee chairman jason chaffetz set political world ablaze tweet  fbi dir informed me   the fbi learned existence emails appear pertinent investigation

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['text'])

word_index=tokenizer.word_index

vocab_size=len(word_index)

2024-05-19 09:41:19.933276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 09:41:19.933424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 09:41:20.099227: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
sent_length=500

sequences=tokenizer.texts_to_sequences(df['text'])
padded_seq=pad_sequences(sequences,maxlen=sent_length,padding='post',truncating='post')

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(padded_seq,df['label'],test_size=0.20,random_state=42,)

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

embedding_vector_features = 40



# Define and build the CNN-LSTM model
model = Sequential()

# Add embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_vector_features, input_length=sent_length))

# Add Conv1D layer
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))

# Add MaxPooling1D layer
model.add(MaxPooling1D(pool_size=2))

# Add Bidirectional LSTM layer
model.add(Bidirectional(LSTM(100)))

# Add Dropout layer
model.add(Dropout(0.3))

# Add Dense output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.build(input_shape=(None, sent_length))
print(model.summary())

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 40)        │     6,083,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 500, 32)        │         3,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 250, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200)            │       106,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,194,393 (23.63 MB)

 Trainable params: 6,194,393 (23.63 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 107s 394ms/step - accuracy: 0.7548 - loss: 0.4415 - val_accuracy: 0.9591 - val_loss: 0.1148
Epoch 2/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 141s 391ms/step - accuracy: 0.9818 - loss: 0.0564 - val_accuracy: 0.9576 - val_loss: 0.1089
Epoch 3/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 103s 396ms/step - accuracy: 0.9974 - loss: 0.0120 - val_accuracy: 0.9600 - val_loss: 0.1494
Epoch 4/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 141s 392ms/step - accuracy: 0.9986 - loss: 0.0071 - val_accuracy: 0.9588 - val_loss: 0.1607
Epoch 5/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 142s 393ms/step - accuracy: 0.9997 - loss: 0.0024 - val_accuracy: 0.9603 - val_loss: 0.1839
Epoch 6/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 143s 397ms/step - accuracy: 0.9966 - loss: 0.0109 - val_accuracy: 0.9400 - val_loss: 0.2661
Epoch 7/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 140s 390ms/step - accuracy: 0.9813 - loss: 0.0591 - val_accuracy: 0.9514 - val_loss: 0.1618
Epoch 8/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 142s 392ms/step - accuracy: 0.9